In [1]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import load_dataset

In [2]:
model = TFAutoModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [29]:
# this inp shd take from user as inp 
inp = ['Hello world', 'Hi how are you']
inputs = tokenizer(inp, padding=True, truncation=True, return_tensors='tf')
inputs

{'input_ids': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[ 101, 7592, 2088,  102,    0,    0],
       [ 101, 7632, 2129, 2024, 2017,  102]])>, 'token_type_ids': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1]])>}

In [21]:
output = model(inputs)
output

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(2, 6, 768), dtype=float32, numpy=
array([[[-0.21088697,  0.11262984, -0.15112709, ..., -0.5123425 ,
          0.00360312,  0.3305137 ],
        [-0.5312121 ,  0.08091751,  0.98387825, ...,  0.0910192 ,
          0.15193745,  0.31607923],
        [-0.81602246, -0.6659761 , -0.08843648, ..., -0.2080327 ,
          0.06849274, -0.5728676 ],
        [ 0.89145756,  0.11419122, -0.18477052, ...,  0.10600633,
         -0.66406184, -0.27207452],
        [-0.4692797 , -0.00563174,  0.55745786, ..., -0.15241298,
          0.02615247,  0.16874756],
        [-0.67426646, -0.23956752,  0.15571368, ..., -0.08268501,
         -0.01054372, -0.03581863]],

       [[ 0.09104545, -0.02801905, -0.12474953, ..., -0.16195123,
          0.10934503,  0.16669227],
        [ 0.4649679 , -0.0777914 ,  0.73076236, ..., -0.63110435,
          0.5957395 , -0.40303573],
        [-0.15863381, -0.6669987 , -0.36142293, ...,  0.23554692

In [6]:
emotions = load_dataset('SetFit/emotion')

Repo card metadata block was not found. Setting CardData to empty.


In [22]:
# tok the ds whole
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [28]:
# setting 'input_ids', 'attention_mask', 'token_type_ids', and 'label'
# to the tensorflow format. Now if you access this dataset you will get these
# columns in `tf.Tensor` format

emotions_encoded.set_format('tf',columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

# setting BATCH_SIZE to 64.
BATCH_SIZE = 64

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split of `emotions_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [24]:
class BERTForClassification(tf.keras.Model):
    
    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')
        
        # softmx for 0/1 op only not in pretrain model 
        
        
    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [25]:
classifier = BERTForClassification(model, num_classes=6)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [26]:
history = classifier.fit(
    train_dataset,
    epochs=1
)

250/250 [==============================] - 4613s 18s/step - loss: 0.9894 - accuracy: 0.6391


In [27]:
classifier.evaluate(test_dataset)

32/32 [==============================] - 143s 4s/step - loss: 0.3698 - accuracy: 0.8740


[0.3698185086250305, 0.8740000128746033]